In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

In [2]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
movie_ratings = pd.merge(movies, ratings, on="movieId")

In [3]:
def get_recommendations(genre, min_rating_threshold, top_N):
    genre_movies = movie_ratings[movie_ratings["genres"].str.contains(genre, case=False, regex=True)]

    movie_stats = genre_movies.groupby("movieId").agg({"rating": ['mean', 'count']})
    movie_stats.columns = ["avg_rating", "num_rating"]

    filtered_movies = movie_stats[(movie_stats["avg_rating"] >= min_rating_threshold) & (movie_stats["num_rating"] >= 1)]

    top_movies = filtered_movies.sort_values(by="avg_rating", ascending=False).head(top_N)

    recommended_movies = pd.merge(top_movies, movies, left_index=True, right_on="movieId")

    return recommended_movies[["title", "avg_rating", "num_rating"]]

In [4]:
genre_input = widgets.Text(placeholder="Enter a genre", description="Genre:")
rating_threshold_input = widgets.FloatLogSlider(min=0, max=5, step=0.1, description="Rating Threshold:")
top_N_input = widgets.IntSlider(min=1, max=50, step=1, description="Top N:")

In [5]:
def update_recommendation(change):
    genre = genre_input.value
    min_rating_threshold = rating_threshold_input.value
    top_N = top_N_input.value

    recommendations = get_recommendations(genre, min_rating_threshold, top_N)

    output.clear_output()
    with output:
        display(recommendations)

In [6]:
search_button = widgets.Button(description="Get Recommendations")
search_button.on_click(update_recommendation)
output = widgets.Output()

layout = widgets.VBox([genre_input, rating_threshold_input, top_N_input, search_button, output])

display(layout)